In [82]:
import json
import tqdm
import hashlib
from typing import (
    Union,
    Sequence
)
import math
import random
import sympy as sp
import glob
import pandas as pd

In [83]:

# function hub
def read_json_file(file_path): 
    ''' opens a json file, return its content save in a dictionary
    @input: file path to the json file
    @output: a dictionary of json file content
    '''
    file = open(file_path, 'r')
    values = json.load(file)
    file.close()
    return values

# main hash function
def hash_elems(*a):
    
    
    h = hashlib.sha256()
    h.update("|".encode("utf-8"))
    
    for x in a:

        if not x:
            # This case captures empty lists and None, nicely guaranteeing that we don't
            # need to do a recursive call if the list is empty. So we need a string to
            # feed in for both of these cases. "None" would be a Python-specific thing,
            # so we'll go with the more JSON-ish "null".
            hash_me = "null"

        elif isinstance(x, str):
            # strings are iterable, so it's important to handle them before the following check
            hash_me = x
        elif isinstance(x, Sequence):
             # The simplest way to deal with lists, tuples, and such are to crunch them recursively.
            hash_me = str(hash_elems(*x))           
        else:
            hash_me = str(x)
        h.update((hash_me + "|").encode("utf-8"))

    # Note: the returned value will range from [1,Q), because zeros are bad
    # for some of the nonces. (g^0 == 1, which would be an unhelpful thing
    # to multiply something with, if you were trying to encrypt it.)

    # Also, we don't need the checked version of int_to_q, because the
    # modulo operation here guarantees that we're in bounds.
    # return int_to_q_unchecked(
    #     1 + (int.from_bytes(h.digest(), byteorder="big") % Q_MINUS_ONE)
    # )

    return (int.from_bytes(h.digest(), byteorder="big") % (q - 1))



def is_valid_of_Zq(n):
    is_valid = True
    if isinstance(n, str):
        n = int(n) 
    if n < 0 or n >= q:
        is_valid = False
 
    return is_valid   


def is_valid_of_Zrp(n):
    is_valid = True
    if isinstance(n, str):
        n = int(n)    
    if n < 0 or n >= p:
        is_valid = False
    
    n_power = pow(n, q, p)    
    if not equals(n_power, 1):
        is_valid = False
    return is_valid


def equals(a, b):
    ''''compares two values
    @input: two integers a, b 
    @output: True if a, b have same values, False otherwise
    '''
    return (a == b)

def get_length(num):
    ''' get the length of an integer using log
    @input: a positive integer
    @output: its length 
    '''
    len_of_num = int(math.log(num, 10) + 1)
    return len_of_num

def mod_p(n):
    if isinstance(n, str):
        n = int(n) 
    return n % p

def get_production(*targets):
    
    product = 1
    for target in targets:
        if isinstance(target, str):
            target = int(target) 
        product = mod_p(product * target)
    return product
    

# the following code is adapted from GeeksforGeeks
def power_mod(x, y, p): 
    ''' do modular exponentiation. 
    @input: x - base, 
            y - exponent,
            m - modulus
    @output: integer result of x^y % m
    '''
    # Initialize result 
    res = 1
      
    # Update x if it is more than or equal to p 
    if(x >= p):
        x = x % p

    # positive exponential
    while (y > 0): 
          
        # If y is odd, multiply x with result 
        if (y & 1): 
            res = (res * x) % p
  
        # y must be even now 
        y = y >> 1; # y = y/2 
        x = (x * x) % p
    return res
    
def __miller_test(d, num):
    ''' find a odd number of d such that num - 1 = d * 2^r
    @input: d - a odd number that num - 1 = d * 2^r for r >= 1
            num - the number needs to be check against
    @output: True if num is prime, False if it's a composite
    '''
    # Pick a random number in [2..n-2] 
    # Corner cases make sure that n > 4 
    a = 2 + random.randint(1, num - 4)

    # Compute a^d % n 
    x = power_mod(a, d, num)
  
    if (x == 1 or x == num - 1): 
        return True
  
    # Keep squaring x while one of the following doesn't happen 
    # (i) d does not reach n-1 
    # (ii) (x^2) % n is not 1 
    # (iii) (x^2) % n is not n-1 
    while (d != num - 1): 
        x = (x * x) % num
        d *= 2
  
        if (x == 1): 
            return False; 
        if (x == num - 1): 
            return True
  
    # Return composite 
    return False

def is_prime(num, k): 
    ''' implements Miller-Rabin algorithm to test the primality of a number
    @input: num - a positive integer
            k - the number of iterations, impacting accuracy
    @output: True if it's a prime, False otherwise 
    '''
    # Corner cases 
    if (num <= 1 or num == 4): 
        return False
    if (num <= 3): 
        return True
  
    # Find r such that n = 2^d * r + 1 for some r >= 1 
    d = num - 1
    while (d % 2 == 0): 
        d //= 2
  
    # Iterate given number of 'k' times 
    for i in range(k): 
        if (__miller_test(d, num) == False): 
            return False
  
    return True

def is_divisor(a, b): 
    '''check if a is a divisor of b
    @input: a, b - positive integers 
    @output: True if a is a divisor of b, False otherwise'''
    return (a % b == 0)

def create_hash_parameter_list(i): 
    '''create a list of parameters needed for hash computation
    @input: i - the i-th trustee/guardian
            base-hash - given base hash code
    @output: a list of all the parameters needed, including base hash Q, Ki,j, hi,j
    '''
    # invalid index input
    if (i < 0 or i > 4): 
        raise ValueError('i should be within range of 0 - 4.')

    else:
        # declare variables
        coefficient_file_path = ''
        param_list = []

        #append base hash to list
        #param_list.append(base_hash)

        # get file name dynamically
        coefficients_file_path = (coefficients_path + str(i) + '.json')
        
        
        # read file 
        coefficients = read_json_file(coefficients_file_path)

        # get all the commitment values Ki,j and append to list 
        commitments = coefficients['coefficient_commitments']
        for commitment in commitments: 
            param_list.append(commitment)
        
        # get all the hi,j values and append to list
        proofs = coefficients['coefficient_proofs']
        for item in proofs:
            h_ij = item['commitment']  # h -> commitment
            param_list.append(h_ij)

        return param_list


### Green box 1.1 Baseline parameters - overview 

In [37]:
# TODO: needs to update paths to new results when the constants.json is given with non-null 


In [84]:

# file path configuration
constants_file_path = 'results/constants.json'
context_file_path = 'new_results/context.json'
ballot_file_path = 'new_results/encrypted_ballots/ballot_ballot-ce63a048-d67c-11ea-8412-acde48001122.json'
ballot_folder_path = 'new_results/encrypted_ballots/'
tally_file_path = 'new_results/tally.json'
coefficients_path = 'new_results/coefficients/coefficient_validation_set_hamilton-county-canvass-board-member-'
description_path = 'new_results/description.json'

constants = read_json_file(constants_file_path)
context = read_json_file(context_file_path)
ballot = read_json_file(ballot_file_path)
tally = read_json_file(tally_file_path)

# basic parameters
g = constants['g']
p = constants['p']
q = constants['q']
r = constants['r']
g_inv = constants['gInv']
K = int(context['elgamal_public_key'])
num_of_guardian = context['number_of_guardians']
threshold = context['quorum']
base_hash = context['crypto_base_hash']
extended_hash = context['crypto_extended_base_hash']

In [85]:
p_expected = int(('''104438888141315250669175271071662438257996424904738378038423348328
3953907971553643537729993126875883902173634017777416360502926082946377942955704498
5420976148418252467735806893983863204397479111608977315510749039672438834271329188
1374801626975452234350528589881677721176191239277291448552115552164104927344620757
8961939840619466145806859275053476560973295158703823395710210329314709715239251736
5523840808458360487786673189314183384224438910259118847234330847012077719019445932
8662497991739135056466263272370300796422984915475619689061525228653308964318490270
6926081744149289517418249153634178342075381874131646013444796894582106870531535803
6662545796026324531037414525697939055519015418561732513850474148403927535855819099
5015804625681054267836812127850996052095762473794291460031064660979266501285839738
1435755902851312071248102599442308951327039250818892493767423329663783709190716162
0235296692173009397831714158082331468230007669177892861540060422814237337064629052
4377485454312723950024587358201266366643058386277816736954760301634424272959224454
4608279405999759391099775667746401633668308698186721172238255007962658564443858927
6348504157753488390520266757856948263869301753031434500465754608438799417919463132
99322976993405829119''').replace('\n', ''))

q_expected = pow(2, 256) - 189

### Green box 1.2 Baseline parameters - check against each election

In [86]:
error = False 

# check p and q
if not equals(p, p_expected):
    error = True 
    print("The actual p value doesn't equal to the expected. ")
if not equals(q, q_expected):
    error = True
    print("The actual q value doesn't equal to the expected. ")
    
# use Miller-Rabin algorithm to check the primality of p and q
# set iteration to run 50 times by default
DEFAULT_K = 50
if not is_prime(p, DEFAULT_K): 
    error = True
    print('''It\'s verfied as {res} that {var} is a prime number.'''
      .format(res = is_prime(p, DEFAULT_K), var = "p"))
if not is_prime(q, DEFAULT_K): 
    error = True
    print('''It\'s verfied as {res} that {var} is a prime number.'''
      .format(res = is_prime(q, DEFAULT_K), var = "q"))

# check equation p - 1 = qr
if not equals(p - 1, q * r): 
    error = True
    print('''It\'s verfied as {res} that {equation}.'''
      .format(res = equals(p - 1, q * r), equation = "p - 1 = q * r"))

# check q is not a divisor of r, and 1 < g < p
if is_divisor(q, r):
    error = True
    print('''It\'s verfied as {res} that {var1} is not a divisor of {var2}.'''
      .format(res = not(is_divisor(q, r)), var1 = "q", var2 = "r"))

# check 1 < g < p
is_within_range = ((g > 1) and (g < p))
if not is_within_range: 
    error = True
    print('''It\'s verfied as {res} that {cond}.'''
      .format(res = is_within_range, cond = "1 < g < p"))

# check g^q mod p = 1 
result_actual = power_mod(g, q, p)
result_expected = 1 
if not equals(result_actual, result_expected):
    error = True
    print('''It\'s verfied as {res} that {equation}.'''
      .format(res = str(equals(result_actual, result_expected)), 
              equation = 'g^q mod p = 1'))
if not error:
    print('[green box 1] success')


[green box 1] success


### Green box 2.1 Key Generation, confirm hash computation
calculate $c_i = H(Q,K_{i,0},K_{i,1},K_{i,2},...,K_{i, k-1},h_{i,0},h_{i,1},h_{i,2},...,h_{i,k-1})$ mod q

related file: context.json

coefficients files


modify to:  $c_{i,j} = H(Q,K_{i,j},h_{i,j})$ mod q

### Green box 2.2 - check: $g^{u_{ij}} mod p = h_{i,j}K^{c_i}_{i,j} mod p$

In [102]:
# unfinished, results are false

error = False
# uses double for-loop to check, 0 <= i < 5, 0 <= j < 3
for i in range(0, num_of_guardian): 
    coefficients = read_json_file(coefficients_path + str(i) + '.json')
    
    for j in range(0, threshold): 
        # get given values
        coefficient_proofs = coefficients['coefficient_proofs'][j]
        u_ij = int(coefficient_proofs['response']) # u, modify to new data field
        h_ij = int(coefficient_proofs['commitment']) # h, modify to new data field
        k_ij = int(coefficient_proofs['public_key'])
        c_ij = int(coefficient_proofs['challenge']) # c
        
        # computed values
        c_ij_computed = hash_elems(base_hash, k_ij, u_ij) % q
        
        # verify c values are computed correctly
        if not equals(c_ij, c_ij_computed):
            error = True
            #print(("The actual c_ij value does not equal to the expected. when i = {i}, j = {j}")
            #      .format(i = i, j = j))
            print("actual   " + str(c_ij))
            print("computed " + str(c_ij_computed))
        box_2_left = pow(g, u_ij, p)
        box_2_right = mod_p(mod_p(h_ij) * pow(k_ij, ci_list[i], p))
        
        # verify both sides of the equation equal to each other
        if not equals(box_2_left, box_2_right):
            error = True
            print("𝑔𝑢𝑖𝑗𝑚𝑜𝑑𝑝 != ℎ𝑖,𝑗𝐾𝑐𝑖𝑖,𝑗𝑚𝑜𝑑𝑝")

        
if not error:
    print("success")

actual   985941757942741440667939151541959156401297201627502171725567595554716972256
computed 103133403323955517559607983769839238443604600398557280880257852012622398933697
𝑔𝑢𝑖𝑗𝑚𝑜𝑑𝑝 != ℎ𝑖,𝑗𝐾𝑐𝑖𝑖,𝑗𝑚𝑜𝑑𝑝
actual   79612685152293585608451307073906463060575499901711780427178381630563209000078
computed 5419634312675361244008982909919970282861449161742879717847158179430853565121
𝑔𝑢𝑖𝑗𝑚𝑜𝑑𝑝 != ℎ𝑖,𝑗𝐾𝑐𝑖𝑖,𝑗𝑚𝑜𝑑𝑝
actual   91636802862839145390912263847428965401692337254074098883996720023002713594319
computed 57267916515944235142852814048554213531124046601424322180540450757958970681716
𝑔𝑢𝑖𝑗𝑚𝑜𝑑𝑝 != ℎ𝑖,𝑗𝐾𝑐𝑖𝑖,𝑗𝑚𝑜𝑑𝑝
actual   105907532015973376090190129394524640797323947580995667174745637885610207031893
computed 34937608199032593216605823192356845223235788865873528224109491989425791638128
𝑔𝑢𝑖𝑗𝑚𝑜𝑑𝑝 != ℎ𝑖,𝑗𝐾𝑐𝑖𝑖,𝑗𝑚𝑜𝑑𝑝
actual   80683151226353167784151752918977020442069234738022658152669636728598377605262
computed 64461804425605098307109450853097178067988067914153141770139124925769407391018
𝑔𝑢𝑖𝑗𝑚𝑜𝑑𝑝 != ℎ𝑖,𝑗𝐾𝑐𝑖𝑖,𝑗𝑚

### green box 3 & 4

In [ ]:
#TODO: flatten all the aggregated "if" statements, keep only one parameter in one statement

In [90]:

# entire ballot files
big_alpha_list = []
big_beta_list = []

for file in glob.glob(ballot_folder_path + "*.json"):
    
    ballot = read_json_file(file)
    error = False
    
    # green box 3
    contests = ballot['contests']
    for contest in contests:

        ballot_selections = contest['ballot_selections']
        for ballot_selection in ballot_selections:


            object_id = ballot_selection['object_id']

            alpha = int(ballot_selection['ciphertext']['pad'])
            beta = int(ballot_selection['ciphertext']['data'])
            big_alpha_list.append(alpha)
            big_beta_list.append(beta)

            a0 = int(ballot_selection['proof']['proof_zero_pad'])
            a1 = int(ballot_selection['proof']['proof_one_pad'])
            b0 = int(ballot_selection['proof']['proof_zero_data'])
            b1 = int(ballot_selection['proof']['proof_one_data'])
            c0 = int(ballot_selection['proof']['proof_zero_challenge'])
            c1 = int(ballot_selection['proof']['proof_one_challenge'])
            v0 = int(ballot_selection['proof']['proof_zero_response'])
            v1 = int(ballot_selection['proof']['proof_one_response'])

            # unused assignments, see if want to delete
            alpha_pow = pow(alpha, q, p)
            beta_pow = pow(beta, q, p)
            a0_pow = pow(a0, q, p)
            b0_pow = pow(b0, q, p)
            a1_pow = pow(a1, q, p)
            b1_pow = pow(b1, q, p)

            #print("a0:"+str(a0))
            #print("alpha_1:"+str(alpha_1))
            # 3.1 The given values alpha, beta, a0, b0, a1, and b1 are all in the set Zpr.


            if not is_valid_of_Zrp(alpha): 
                error = True
                print ("alpha is not in set Zrp.")
            
            if not is_valid_of_Zrp(beta): 
                error = True
                print ("beta is not in set Zrp.")
            
            if not is_valid_of_Zrp(a0): 
                error = True
                print ("a0 is not in set Zrp.")
                
            if not is_valid_of_Zrp(a1): 
                error = True
                print ("a1 is not in set Zrp.")
            
            if not is_valid_of_Zrp(b0): 
                error = True
                print ("b0 is not in set Zrp.")
                
            if not is_valid_of_Zrp(b1): 
                error = True
                print ("b1 is not in set Zrp.")

            # 3.2 The challenge c is computed as c = H(Q, (alpha, beta), (a0, b0), (a1, b1))
            # tempString = str(q_bar) + str(alpha) + str(beta) + str(a0) + str(b0) + str(a1) + str(b1)
            # tempList = (q_bar, (alpha, beta), (a0, b0), (a1, b1))
            c = hash_elems(extended_hash, alpha, beta, a0, b0, a1, b1)
            #print(c)

            #print((c0 + c1) % q)

            if not equals(c, (c0 + c1) % q):
                error = True
                print("c != (c0 + c1) % q.")

            # 3.3 The given values c0, c1, v0, and v1 are each in the set Zq.
            if not is_valid_of_Zq(c0):
                error = True 
                print("c0 is not in the set of Zq. ")
               
            if not is_valid_of_Zq(c1):
                error = True 
                print("c1 is not in the set of Zq. ")
                
            if not is_valid_of_Zq(v0):
                error = True
                print("v0 is not in the set of Zq. ")
                
            if not is_valid_of_Zq(v1):
                error = True
                print("v1 is not in the set of Zq. ")

            # 1st equation: gv0=a0alphac0 mod p
            g_v0_ls = pow(g, v0, p)
            g_v0_rs = mod_p(a0 * pow(alpha, c0, p))

            # 2nd equation: gv1=a1alphac1 mod p
            g_v1_ls = pow(g, v1, p)
            g_v1_rs = mod_p(a1 * pow(alpha, c1, p))

            # 3rd equation: Kv0=b0betac0 mod p
            K_v0_ls = pow(K, v0, p)
            K_v0_rs = mod_p(b0 * pow(beta, c0, p))

            # 4th equation: gc1Kv1=b1betac1 mod p
            fourth_ls = mod_p(pow(g, c1, p) * pow(K, v1, p))
            fourth_rs = mod_p(b1 * pow(beta, c1, p))

            # checking
            if not equals(g_v0_ls,g_v0_rs):
                print("g_vo != a_0*alpha^c_0 at cast_ballots {i}, contest {j}, selection {k}")
                error = True
            if not equals(g_v1_ls, g_v1_rs):
                print("g_v1 != a_1*alpha^c_1")
                error = True
            if not equals(K_v0_ls, K_v0_rs):
                print("K_v0 != b_0*alpha^c_0")
                error = True
            if not equals(fourth_ls, fourth_rs):
                print("g_c1 * K_v1 != (b1 * beta^c1) % p")
                error = True


    if not error:
        print("[green box 3 success] " + file)  
    
    # green box 4
    for contest in contests:

        # total parameters
        big_a = int(contest['proof']['pad'])
        big_b = int(contest['proof']['data'])
        big_c = int(contest['proof']['challenge'])
        big_v = int(contest['proof']['response'])
        big_constant = int(contest['proof']['constant'])
        big_alpha = 1
        big_beta = 1

        if not is_valid_of_Zq(big_v):
            error = True
            print("V is not in the set of Zq. ")
        
        if not is_valid_of_Zrp(big_a):
            error = True
            print ("A is not in the set of Zrp. ")
            
        if not is_valid_of_Zrp(big_b):
            error = True
            print("B is not in the set of Zrp. ")

        ballot_selections = contest['ballot_selections']
        for ballot_selection in ballot_selections:

            alpha = int(ballot_selection['ciphertext']['pad'])
            beta = int(ballot_selection['ciphertext']['data'])  
            big_alpha = mod_p(big_alpha * alpha)
            big_beta = mod_p(big_beta * beta)

        temp_c = hash_elems(extended_hash, big_alpha, big_beta, big_a, big_b)
        #print(temp_c)
        #print(big_c)


        # g power v mod p = a * A power c mod p
        g_v_ls = pow(g, big_v, p)
        a_ac_rs = mod_p((mod_p(big_a)) * pow(big_alpha, big_c, p))

        #print(g_v_ls)
        #print(a_ac_rs)
        if not equals(g_v_ls, a_ac_rs):
            error = True
            print("g_v_ls does not equal to a_ac_rs.")


        if big_constant is None:
            constant_q = 0
        else:
            constant_q = big_constant

        # g power Lc mod p * K power v mod p = b * B power c mod p
        g_l_k_ls = mod_p(pow(g, mod_p(constant_q * big_c), p) * pow(K, big_v, p))
        b_bc_rs = mod_p(mod_p(big_b) * pow(big_beta, big_c, p))


        # mult_p(g_pow_p(mult_p(c, constant_q)), pow_p(k, v))
        # mult_p(b, pow_p(beta, c))
        #print(g_l_k_ls)
        #print(b_bc_rs)
        if not equals(g_l_k_ls, b_bc_rs):
            error = True
            
            print("g_l_k_ls does not equal to b_bc_rs.")        

    if not error:
        print("[green box 4 success] " + file)
    
    

[green box 3 success] new_results/encrypted_ballots/ballot_ballot-ce65aa50-d67c-11ea-8412-acde48001122.json
[green box 4 success] new_results/encrypted_ballots/ballot_ballot-ce65aa50-d67c-11ea-8412-acde48001122.json
[green box 3 success] new_results/encrypted_ballots/ballot_ballot-ce66267e-d67c-11ea-8412-acde48001122.json
[green box 4 success] new_results/encrypted_ballots/ballot_ballot-ce66267e-d67c-11ea-8412-acde48001122.json
[green box 3 success] new_results/encrypted_ballots/ballot_ballot-ce658e12-d67c-11ea-8412-acde48001122.json
[green box 4 success] new_results/encrypted_ballots/ballot_ballot-ce658e12-d67c-11ea-8412-acde48001122.json
[green box 3 success] new_results/encrypted_ballots/ballot_ballot-ce644570-d67c-11ea-8412-acde48001122.json
[green box 4 success] new_results/encrypted_ballots/ballot_ballot-ce644570-d67c-11ea-8412-acde48001122.json
[green box 3 success] new_results/encrypted_ballots/ballot_ballot-ce68fa66-d67c-11ea-8412-acde48001122.json
[green box 4 success] new_re

KeyboardInterrupt: 

In [110]:

# one sample data
ballot = read_json_file(ballot_file_path)

error = False

contests = ballot['contests']
for contest in contests:
    
    ballot_selections = contest['ballot_selections']
    for ballot_selection in ballot_selections:


        objectId = ballot_selection['object_id']

        alpha = int(ballot_selection['ciphertext']['pad'])
        beta = int(ballot_selection['ciphertext']['data'])

        a0 = int(ballot_selection['proof']['proof_zero_pad'])
        a1 = int(ballot_selection['proof']['proof_one_pad'])
        b0 = int(ballot_selection['proof']['proof_zero_data'])
        b1 = int(ballot_selection['proof']['proof_one_data'])
        c0 = int(ballot_selection['proof']['proof_zero_challenge'])
        c1 = int(ballot_selection['proof']['proof_one_challenge'])
        v0 = int(ballot_selection['proof']['proof_zero_response'])
        v1 = int(ballot_selection['proof']['proof_one_response'])

        # unused assignments, see if want to delete
        alpha_pow = pow(alpha, q, p)
        beta_pow = pow(beta, q, p)
        a0_pow = pow(a0, q, p)
        b0_pow = pow(b0, q, p)
        a1_pow = pow(a1, q, p)
        b1_pow = pow(b1, q, p)

        #print("a0:"+str(a0))
        #print("alpha_1:"+str(alpha_1))
        # 3.1 The given values alpha, beta, a0, b0, a1, and b1 are all in the set Zpr.
        
        
        if not is_valid_of_Zrp(alpha): 
            error = True
            print ("alpha is not in set Zrp.")
            
        if not is_valid_of_Zrp(beta): 
            error = True
            print ("beta is not in set Zrp.")
            
        if not is_valid_of_Zrp(a0): 
            error = True
            print ("a0 is not in set Zrp.")
                
        if not is_valid_of_Zrp(a1): 
            error = True
            print ("a1 is not in set Zrp.")
            
        if not is_valid_of_Zrp(b0): 
            error = True
            print ("b0 is not in set Zrp.")
                
        if not is_valid_of_Zrp(b1): 
            error = True
            print ("b1 is not in set Zrp.")
            
        # 3.2 The challenge c is computed as c = H(Q, (alpha, beta), (a0, b0), (a1, b1))
        # tempString = str(q_bar) + str(alpha) + str(beta) + str(a0) + str(b0) + str(a1) + str(b1)
        # tempList = (q_bar, (alpha, beta), (a0, b0), (a1, b1))
        c = hash_elems(extended_hash, alpha, beta, a0, b0, a1, b1)
        #print(c)
        
        #print((c0 + c1) % q)
        
        if not equals(c, (c0 + c1) % q):
            print("c != (c0 + c1) % q.")
            error = True

        # 3.3 The given values c0, c1, v0, and v1 are each in the set Zq.
        if not is_valid_of_Zq(c0):
            error = True 
            print("c0 is not in the set of Zq. ")
               
        if not is_valid_of_Zq(c1):
            error = True 
            print("c1 is not in the set of Zq. ")
                
        if not is_valid_of_Zq(v0):
            error = True
            print("v0 is not in the set of Zq. ")
                
        if not is_valid_of_Zq(v1):
            error = True
            print("v1 is not in the set of Zq. ")        

        # 1st equation: gv0=a0alphac0 mod p
        g_v0_ls = pow(g, v0, p)
        g_v0_rs = mod_p(a0 * pow(alpha, c0, p))
    
        # 2nd equation: gv1=a1alphac1 mod p
        g_v1_ls = pow(g, v1, p)
        g_v1_rs = mod_p(a1 * pow(alpha, c1, p))

        # 3rd equation: Kv0=b0betac0 mod p
        K_v0_ls = pow(K, v0, p)
        K_v0_rs = mod_p(b0 * pow(beta, c0, p))
    
        # 4th equation: gc1Kv1=b1betac1 mod p
        fourth_ls = mod_p(pow(g, c1, p) * pow(K, v1, p))
        fourth_rs = mod_p(b1 * pow(beta, c1, p))

        # checking
        if not equals(g_v0_ls,g_v0_rs):
            print("g_vo != a_0*alpha^c_0 at cast_ballots {i}, contest {j}, selection {k}")
            error = True
        if not equals(g_v1_ls, g_v1_rs):
            print("g_v1 != a_1*alpha^c_1")
            error = True
        if not equals(K_v0_ls, K_v0_rs):
            print("K_v0 != b_0*alpha^c_0")
            error = True
        if not equals(fourth_ls, fourth_rs):
            print("g_c1 * K_v1 != (b1 * beta^c1) % p")
            error = True

                                               
if not error:
    print("success")   



success


### TODO: (box 4)
1. modify variable names -> make sure to use underline instead of camal cases
2. change function names accordingly
3. flatten if statements

In [19]:
# Green box 4

for file in glob.glob(ballot_folder_path + "*.json"):
    
    ballot = read_json_file(file)

    error = False

    contests = ballot['contests']
    for contest in contests:

        # total parameters
        big_a = int(contest['proof']['pad'])
        big_b = int(contest['proof']['data'])
        big_c = int(contest['proof']['challenge'])
        big_v = int(contest['proof']['response'])
        big_constant = int(contest['proof']['constant'])
        big_alpha = 1
        big_beta = 1

        if not is_valid_of_Zq(big_v):
            error = True
            print("V is not in the set of Zq. ")
        
        if not is_valid_of_Zrp(big_a):
            error = True
            print ("A is not in the set of Zrp. ")
            
        if not is_valid_of_Zrp(big_b):
            error = True
            print("B is not in the set of Zrp. ")

        ballot_selections = contest['ballot_selections']
        for ballot_selection in ballot_selections:

            alpha = int(ballot_selection['ciphertext']['pad'])
            beta = int(ballot_selection['ciphertext']['data'])  
            big_alpha = mod_p(big_alpha * alpha)
            big_beta = mod_p(big_beta * beta)

        temp_c = hash_elems(extended_hash, big_alpha, big_beta, big_a, big_b)
        #print(temp_c)
        #print(big_c)


        # g power v mod p = a * A power c mod p
        g_v_ls = pow(g, big_v, p)
        a_ac_rs = mod_p((mod_p(big_a)) * pow(big_alpha, big_c, p))

        #print(g_v_ls)
        #print(a_ac_rs)
        if g_v_ls != a_ac_rs:
            error = True
            print("g_v_ls is not equal a_ac_rs.")


        if big_constant is None:
            constant_q = 0
        else:
            constant_q = big_constant

        # g power Lc mod p * K power v mod p = b * B power c mod p
        g_l_k_ls = mod_p(pow(g, mod_p(constant_q * big_c), p) * pow(K, big_v, p))
        b_bc_rs = mod_p(mod_p(big_b) * pow(big_beta, big_c, p))


        # mult_p(g_pow_p(mult_p(c, constant_q)), pow_p(k, v))
        # mult_p(b, pow_p(beta, c))
        #print(g_l_k_ls)
        #print(b_bc_rs)
        if g_l_k_ls != b_bc_rs:
            error = True
            print("g_l_k_ls is not equal b_bc_rs.")        

    if not error:
        print("success")   

success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success


### green box 6 - decryption

In [91]:
# TODO: verify big_A, big_B by getting products from 100 ballot files

# create data frames with column names 

selection_name_lists = []

def switch_idx(contest_name): 
    switcher = {
        'president-vice-president-contest': 0,
        'ozark-governor': 1,
        'congress-district-5-contest': 2,
        'congress-district-7-contest': 3,
        'pismo-beach-school-board-contest': 4,
        'somerset-school-board-contest': 5,
        'arlington-chief-justice-retain-demergue': 6,
        'exeter-utility-district-referendum-contest': 7,
    }
    
    if (contest_name in switcher.keys()): 
        return switcher[contest_name]
    else: 
        return None


description = read_json_file(description_path)
contests = description['contests']

for i in range(len(contests)):
    selection_name_lists.append(['id'])
    
for i in range(len(contests)):
    contest = contests[i]
    contest_name = contest['object_id']
    selections = contest['ballot_selections']
    curr_list = selection_name_lists[switch_idx(contest_name)]
    for selection in selections: 
        selection_name = selection['object_id']
        if not selection_name in curr_list: 
            curr_list.append(selection_name + '-a')
            curr_list.append(selection_name + '-b')

In [92]:
print(selection_name_lists)

[['id', 'barchi-hallaren-selection-a', 'barchi-hallaren-selection-b', 'cramer-vuocolo-selection-a', 'cramer-vuocolo-selection-b', 'court-blumhardt-selection-a', 'court-blumhardt-selection-b', 'boone-lian-selection-a', 'boone-lian-selection-b', 'hildebrand-garritty-selection-a', 'hildebrand-garritty-selection-b', 'patterson-lariviere-selection-a', 'patterson-lariviere-selection-b', 'write-in-selection-president-a', 'write-in-selection-president-b'], ['id', 'franz-selection-a', 'franz-selection-b', 'harris-selection-a', 'harris-selection-b', 'bargmann-selection-a', 'bargmann-selection-b', 'abcock-selection-a', 'abcock-selection-b', 'steel-loy-selection-a', 'steel-loy-selection-b', 'sharp-selection-a', 'sharp-selection-b', 'walace-selection-a', 'walace-selection-b', 'williams-selection-a', 'williams-selection-b', 'alpern-selection-a', 'alpern-selection-b', 'windbeck-selection-a', 'windbeck-selection-b', 'sharp-althea-selection-a', 'sharp-althea-selection-b', 'greher-selection-a', 'greher-

In [93]:
dfs = []

for i in range(len(selection_name_lists)):
    df = pd.DataFrame(columns = selection_name_lists[i])
    dfs.append(df)

# read all ballot files
# unfinished
for file in glob.glob(ballot_folder_path + "*.json"):
    ballot = read_json_file(file)
    ballot_id = ballot['object_id']
    ballot_state = ballot['state']
    contests = ballot['contests']
    
    # ignore spoiled ballots
    if (ballot_state == 'CAST'): 
        for i in range(len(contests)):
            contest = contests[i]
            contest_name = contest['object_id']
            curr_df = dfs[switch_idx(contest_name)]
            selections = contest['ballot_selections']
            # store data from a ballot
            curr_list = []
            curr_list.append(ballot_id)
            for selection in selections:
                is_placeholder = selection['is_placeholder_selection']
                if not is_placeholder: 
                    selection_name = selection['object_id']
                    a = selection['ciphertext']['pad']
                    b = selection['ciphertext']['data']
                    curr_list.append(a)
                    curr_list.append(b)
            # assign to a row and add to a designated dataframe
            curr_df_leng = len(curr_df)
            curr_df.loc[curr_df_leng] = curr_list
                    

In [94]:
# check number of cast ballots of each contest
for df in dfs:
    print(df.shape)

(92, 15)
(92, 55)
(34, 13)
(58, 13)
(16, 21)
(17, 17)
(45, 5)
(11, 5)


In [95]:
def get_product(dataframe, column_index):
    product = 1
    column = dataframe.iloc[:, column_index]
    
    for i in range(len(column)):

        product *= int(column.loc[i])
        
    return product

In [96]:
# verify A = alpha_j product, B = beta_j product

contest_names = list(tally.keys())
# loop through contests
for contest_name in contest_names:
    error = False

    # match tally with specific dataframe
    curr_df = dfs[switch_idx(contest_name)]
    
     # get all the selection names in a contest
    selection_names = list(tally[contest_name]['selections'].keys())
    
    idx = 0
    
    # loop through selections
    for selection_name in selection_names:
        selection = tally[contest_name]['selections'][selection_name]
        # get A and B
        big_alpha = selection['message']['pad']
        big_beta= selection['message']['data']
        # get alpha_j and beta_j products
        alpha_j_product = get_product(curr_df, 2 * idx + 1)
        beta_j_product = get_product(curr_df, 2 * idx + 2)
        if not equals(alpha_j_product, big_alpha):
            error = True
            #print(alpha_j_product)
            #print('\n\'' + str(big_alpha))
            print(selection_name + " Alpha error")
        if not equals(beta_j_product, big_beta):
            error = True
            print(selection_name + " Beta error")
        if equals(alpha_j_product, big_alpha) and equals(beta_j_product, big_beta):
            print(selection_name + " sucess")
        idx += 1
        
if not error:
    print("success")

barchi-hallaren-selection Alpha error
barchi-hallaren-selection Beta error
cramer-vuocolo-selection Alpha error
cramer-vuocolo-selection Beta error
court-blumhardt-selection Alpha error
court-blumhardt-selection Beta error
boone-lian-selection Alpha error
boone-lian-selection Beta error
hildebrand-garritty-selection Alpha error
hildebrand-garritty-selection Beta error
patterson-lariviere-selection Alpha error
patterson-lariviere-selection Beta error
write-in-selection-president Alpha error
write-in-selection-president Beta error
franz-selection Alpha error
franz-selection Beta error
harris-selection Alpha error
harris-selection Beta error
bargmann-selection Alpha error
bargmann-selection Beta error
abcock-selection Alpha error
abcock-selection Beta error
steel-loy-selection Alpha error
steel-loy-selection Beta error
sharp-selection Alpha error
sharp-selection Beta error
walace-selection Alpha error
walace-selection Beta error
williams-selection Alpha error
williams-selection Beta error

In [66]:
# green box 6

# get all ki_0
k_i0_list = []
for i in range(0, num_of_guardian): 
    coefficients = read_json_file(coefficients_path + str(i) + '.json')
    k_i0 = int(coefficients['coefficient_commitments'][0])
    k_i0_list.append(k_i0)

#for file in glob.glob(ballot_folder_path + "*.json"):
    
# get all the contest names stored in the first layer
contest_names = list(tally.keys())

# loop over each contest 
for contest_name in contest_names: 
    error = False
    print(contest_name)
    
    # get all the selection names in a contest
    selection_names = list(tally[contest_name]['selections'].keys())
    
    # loop over each selection within a contest
    for selection_name in selection_names:
        print("     " + selection_name)
        selection = tally[contest_name]['selections'][selection_name]
        shares = selection['shares']
        big_A = int(selection['message']['pad'])
        big_B = int(selection['message']['data'])
        
        # loop over each guardian's share of decryption
        for i in range(num_of_guardian):
            share = shares[i]
            M_i = int(share['share'])
            proof = share['proof']
            v_i = int(proof['response'])
            a_i = int(proof['pad'])
            b_i = int(proof['data'])
            c_i = int(proof['challenge'])
            k_i = k_i0_list[i]
            
            # calculate using given values
            c_i_expected = hash_elems(extended_hash, big_A, big_B, a_i, b_i, M_i)
            #print("actual c   " + str(c_i))
            #print("expected c " + str(c_i_expected))
            equ_1_left = pow(g, v_i, p)
            equ_1_right = mod_p(mod_p(a_i) * pow(k_i, c_i, p))
            equ_2_left = pow(big_A, v_i, p)
            equ_2_right = mod_p(mod_p(b_i) * pow(M_i, c_i, p))
            #print("equation1 left   " + str(equ_1_left))
            #print("equation1 right   " + str(equ_1_right))
            
            # confirm that v_i is in the set of Z_q
            if not is_valid_of_Zq(v_i):
                error = True
                print("       v_i error " + str(error))
                
            # confirm that a_i and b_i are both in the set Z_r_q
            if not is_valid_of_Zrp(a_i):
                error = True
                print("       a_i error " + str(error))
            if not is_valid_of_Zrp(b_i):
                error = True
                print("       b_i error " + str(error))
                
            # confirm challege value c_i = H(q_bar, (A,B), (a_i, b_i), M_i)
            if not equals(c_i, c_i_expected):
                error = True
                print("       c_i error " + str(error))
                
            # check first equation, g^v_i = a_i * K_i^c_i
            if not equals(equ_1_left, equ_1_right):
                error = True
                print("       equation1 error " + str(error))
                
            # check second equation, A^v_i = b_i * M_i^c_i
            if not equals(equ_2_left, equ_2_right):
                error = True
                print("       equation2 error " + str(error))
                
            
        
if not error:
    print("success")

president-vice-president-contest
     barchi-hallaren-selection
     cramer-vuocolo-selection
     court-blumhardt-selection
     boone-lian-selection
     hildebrand-garritty-selection
     patterson-lariviere-selection
     write-in-selection-president
ozark-governor
     franz-selection
     harris-selection
     bargmann-selection
     abcock-selection
     steel-loy-selection
     sharp-selection
     walace-selection
     williams-selection
     alpern-selection
     windbeck-selection
     sharp-althea-selection
     greher-selection
     alexander-selection
     mitchell-selection
     lee-selection
     ash-selection
     kennedy-selection
     jackson-selection
     brown-selection
     teller-selection
     ward-selection
     murphy-selection
     newman-selection
     callanann-selection
     york-selection
     chandler-selection
     write-in-selection-governor
congress-district-5-contest
     soliz-selection
     keller-selection
     rangel-selection
     argent-select

In [21]:


70994832482198157778563778077321339890089723449913247716198644394282333863496
76399268070314868948319433133099194084979953104525111530814451833587822877544
89915174162554549280401364803765952977396227696680226394538786049235446808187


### green box 9 - tally verification

In [75]:
# green box 9 
error = False

for contest_name in contest_names: 
    # get all the selection names in a contest
    selection_names = list(tally[contest_name]['selections'].keys())
    #print(contest_name) 
    
    # loop over each selection within a contest
    for selection_name in selection_names:
        #print("     " + selection_name)
        m_product = 1
        selection = tally[contest_name]['selections'][selection_name]
        shares = selection['shares']
        big_B = int(selection['message']['data'])
        big_M = int(selection['value'])
        t = int(selection['tally']) 
        
        for share in shares: 
            M_i = int(share['share'])
            m_product *= mod_p(M_i)
            
        # check equation B = M * (M_i products) mod p
        if not equals(mod_p(big_B), mod_p(big_M * m_product)):
            error = True
            print(error)
            
        # check equation M = g^t mod p
        if not equals(big_M, pow(g, t, p)):
            error = True
            print(error)
if not error:
    print("success")

success


### green box 10